<div style="position: absolute; top: 0; left: 0; font-family: 'Garamond'; font-size: 14px;">
    <a href="https://github.com/patriciaapenat" style="text-decoration: none; color: inherit;">Patricia Peña Torres</a>
</div>

<div align="center" style="font-family: 'Garamond'; font-size: 48px;">
    <strong>Proyecto final, BRFSS-clustering</strong>
</div>

<div align="center" style="font-family: 'Garamond'; font-size: 36px;">
    <strong>Imputación de valores nulos</strong>
</div>

__________________

<div style="font-family: 'Garamond'; font-size: 14px;">

Este notebook aborda la imputación de valores nulos, un paso crítico en el preprocesamiento de datos para análisis de clustering. Se utilizan técnicas específicas para tratar valores nulos basándose en las características de cada columna, incluyendo la imputación basada en la media y la mediana, y se aplican condiciones personalizadas para determinar cómo se deben imputar ciertos valores, reflejando la complejidad y la especificidad de los datos de salud pública.


Se concluye con la imputación de los datos y la exportación del DataFrame resultante para su análisis posterior. Este flujo de trabajo detallado y meticuloso muestra la aplicación de Spark para el análisis de grandes volúmenes de datos.
</div>

<div style="font-family: 'Garamond'; font-size: 24px;">
    <strong>Importación de paquetes</strong>
</div>

In [1]:
import pandas as pd
import findspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, DataFrame, functions as F, Window
from pyspark.sql.functions import col, mean, when, lit, monotonically_increasing_id
from pyspark.ml.feature import Imputer

<div style="font-family: 'Garamond'; font-size: 24px;">
    <strong>Configuración de Spark</strong>
</div>

In [2]:
# Si hay un SparkContext existente, debemos cerrarlo antes de crear uno nuevo
if 'sc' in locals() and sc:
    sc.stop()  # Detener el SparkContext anterior si existe

# Configuración de Spark
conf = (
    SparkConf()
    .setAppName("Proyecto_PatriciaA_Peña")  # Nombre de la aplicación en Spark
    .setMaster("local[2]")  # Modo local con un hilo para ejecución
    .set("spark.driver.host", "127.0.0.1")  # Dirección del host del driver
    .set("spark.executor.heartbeatInterval", "3600s")  # Intervalo de latido del executor
    .set("spark.network.timeout", "7200s")  # Tiempo de espera de la red
    .set("spark.executor.memory", "14g")  # Memoria asignada para cada executor
    .set("spark.driver.memory", "14g")  # Memoria asignada para el driver
)

# Crear un nuevo SparkContext con la configuración especificada
sc = SparkContext(conf=conf)

# Configuración de SparkSession (interfaz de alto nivel para trabajar con datos estructurados en Spark)
spark = (
    SparkSession.builder
    .appName("Proyecto_PatriciaA_Peña")  # Nombre de la aplicación en Spark
    .config("spark.sql.repl.eagerEval.enabled", True)  # Habilitar la evaluación perezosa en Spark SQL REPL
    .config("spark.sql.repl.eagerEval.maxNumRows", 1000)  # Número máximo de filas a mostrar en la evaluación perezosa
    .getOrCreate()  # Obtener la sesión Spark existente o crear una nueva si no existe
) 

<div style="font-family: 'Garamond'; font-size: 16px;">
    <strong>Lectura del archivo</strong>
</div>

In [3]:
# Lee el archivo CSV 
df = spark.read.csv(r"C:\\Users\\patri\\OneDrive - UAB\\Documentos\\GitHub\\BRFSS-clustering\\datos\\BRFSS_Cleaner_2022.csv", header=True, inferSchema=True)

In [4]:
# Convierte todas las columnas a tipo double
df = df.select(*[col(column_name).cast("double").alias(column_name) for column_name in df.columns])

<div style="font-family: 'Garamond'; font-size: 20px;">
    <strong>Imputación de nulos</strong>
</div>

<div style="font-family: 'Garamond'; font-size: 14px;">
    <normal>
Trabajamos con un dataset que como hemos comentado proviene de encuestas, así que hay una enorme cantidad de valores nulos, esto lo abordaremos columna a columna, fue necesario hacer una revisión minuciosa del dataset y del codebook para establecer las necesidades específicas que podía tener cada columna, este proceso fue lento pero necesario para poder contar con datos limpios y sin datos nulos.</normal> 
</div>

In [5]:
# Lista de valores a considerar como NA
na_values = [9, 99, 88]  # Esta es la clave para "Refused"

# Marcar valores 9 o 99 como NA en todas las columnas (excepto _AGE80, _FMONTH y _STATE)
for col_name in df.columns:
    if col_name not in ["_AGE80", "_FMONTH", "_STATE"]: 
        df = df.withColumn(col_name, when(col(col_name).isin(na_values), None).otherwise(col(col_name)))

In [6]:
conditions = { # Había muchas más secciones como por ejemplo determinantes sociales, dificultades en la infancia, consumo de cannabis, etc.
               # así mismo había muchas más columnas de secciones como salud respiratoria pero al ser columnas que no tenían condiciones especiales
               # se trataron conjunto a otras con las misma necesidades
    # Salud ginecológica
    "HADMAM": col("_SEX") == 1,
    "HOWLONG": col("_SEX") == 1,
    "CERVSCRN": col("_SEX") == 1,
    "CRVCLCNC": col("_SEX") == 1,
    "CRVCLPAP": col("_SEX") == 1,
    "CRVCLHPV": col("_SEX") == 1,
    "HADHYST2": col("_SEX") == 1,
    "PREGNANT": col("_SEX") == 1,
    # Cáncer colorectal
    'HADSIGM4': col("_AGE80") < 45,
    'COLNSIGM': (col("_AGE80") < 45) | (col("HADSIGM4").isin([1])) | (col("COLNSIGM").isin([1, 7, 9, None])),
    'LASTSIG4': (col("_AGE80") < 45) | (col("HADSIGM4").isin([2, 7, 9, None])) | (col("COLNSIGM").isin([9, None])) | (col("SIGMTES1").isNotNull()),
    'COLNCNCR': col("_AGE80") < 45,
    'VIRCOLO1': (col("_AGE80") < 45) | (col("COLNCNCR").isin([2, 7, 9, None])),
    'VCLNTES2': (col("_AGE80") < 45) | (col("COLNCNCR").isin([2, 7, 9, None])) | (col("VIRCOLO1").isin([2, 7, 9])),
    'SMALSTOL': (col("_AGE80") < 45) | (col("COLNCNCR").isin([2, 7, 9, None])),
    'STOLTEST': (col("_AGE80") < 45) | (col("COLNCNCR").isin([2, 7, 9, None])) | (col("SMALSTOL").isin([2, 7, 9, None])),
    'STOOLDN2': (col("_AGE80") < 45) | (col("COLNCNCR").isin([2, 7, 9, None])),
    'BLDSTFIT': (col("_AGE80") < 45) | (col("COLNCNCR").isin([2, 7, 9, None])) | (col("STOOLDN2").isin([2, 7, 9, None])),
    'SDNATES1': (col("_AGE80") < 45) | (col("COLNCNCR").isin([2, 7, 9, None])) | (col("STOOLDN2").isin([2, 7, 9, None])),
    # Hábito tabáquico
    'LCSFIRST': col('SMOKE100').isin([2, 7, 9]) | col('SMOKDAY2').isin([7, 9]),
    'LCSLAST': col('SMOKE100').isin([2, 7, 9]) | col('SMOKDAY2').isin([7, 9]) | col('LCSFIRST').isin([888]),
    'LCSNUMCG': col('SMOKE100').isin([2, 7, 9]) | col('SMOKDAY2').isin([7, 9]) | col('LCSFIRST').isin([888]),
    'LCSSCNCR': col('LCSCTSC1').isin([2, 7, 9]),
    'LCSCTWHN': col('LCSCTSC1').isin([2, 7, 9]) | col('LCSSCNCR').isin([2, 7, 9]),
    'STOPSMK2': (col('SMOKE100').isin([2, 7, 9])) | (col('SMOKDAY2').isin([1, 2, 7, 9])),
    'LASTSMK2': (col('SMOKE100').isin([2, 7, 9])) | (col('SMOKDAY2').isin([1, 2, 7, 9])) | (col('LASTSMK2').isNull()),
    'MENTCIGS': col('SMOKDAY2').isin([1, 2, 7, 9]),
    'MENTECIG': (col('SMOKDAY2').isin([1, 2, 7, 9])) | (col('ECIGNOW2').isin([1, 4, 7, 9])),
    # Cáncer, sobrevivencia y manejo del dolor 
    'CNCRDIFF': col('CHCSCNC1').isin([2, 7, 9]) | col('CHCOCNC1').isin([2, 7, 9]) | col('CHCSCNC1').isNull() | col('CHCOCNC1').isNull(),
    'CSRVTRT3': col('CHCSCNC1').isin([2, 7, 9]) | col('CHCOCNC1').isin([2, 7, 9]) | col('CHCSCNC1').isNull() | col('CHCOCNC1').isNull(),
    'CSRVPAIN': col('CHCSCNC1').isin([2, 7, 9]) | col('CHCOCNC1').isin([2, 7, 9]) | col('CHCSCNC1').isNull() | col('CHCOCNC1').isNull(),
    'CSRVCTL2': col('CHCSCNC1').isin([2, 7, 9]) | col('CHCOCNC1').isin([2, 7, 9]) | col('CHCSCNC1').isNull() | col('CHCOCNC1').isNull() | col('CSRVPAIN').isin([2, 7, 9]) | col('CSRVPAIN').isNull(),
    'CSRVDOC1': col('CHCSCNC1').isin([2, 7, 9]) | col('CHCOCNC1').isin([2, 7, 9]) | col('CHCSCNC1').isNull() | col('CHCOCNC1').isNull() | col('CSRVTRT3').isin([1, 3, 4, 5, 7, 9]) | col('CSRVTRT3').isNull(),
    'CSRVSUM': col('CHCSCNC1').isin([2, 7, 9]) | col('CHCOCNC1').isin([2, 7, 9]) | col('CHCSCNC1').isNull() | col('CHCOCNC1').isNull() | col('CSRVTRT3').isin([1, 3, 4, 5, 7, 9]) | col('CSRVTRT3').isNull(),
    'CSRVRTRN': col('CHCSCNC1').isin([2, 7, 9]) | col('CHCOCNC1').isin([2, 7, 9]) | col('CHCSCNC1').isNull() | col('CHCOCNC1').isNull() | col('CSRVTRT3').isin([1, 3, 4, 5, 7, 9]) | col('CSRVTRT3').isNull(),
    'CSRVINST': col('CHCSCNC1').isin([2, 7, 9]) | col('CHCOCNC1').isin([2, 7, 9]) | col('CHCSCNC1').isNull() | col('CHCOCNC1').isNull() | col('CSRVTRT3').isin([1, 3, 4, 5, 7, 9]) | col('CSRVTRT3').isNull(),
    'CSRVINSR': col('CHCSCNC1').isin([2, 7, 9]) | col('CHCOCNC1').isin([2, 7, 9]) | col('CHCSCNC1').isNull() | col('CHCOCNC1').isNull() | col('CSRVTRT3').isin([1, 3, 4, 5, 7, 9]) | col('CSRVTRT3').isNull(),
    'CSRVDEIN': col('CHCSCNC1').isin([2, 7, 9]) | col('CHCOCNC1').isin([2, 7, 9]) | col('CHCSCNC1').isNull() | col('CHCOCNC1').isNull() | col('CSRVTRT3').isin([1, 3, 4, 5, 7, 9]) | col('CSRVTRT3').isNull(),
    'CSRVCLIN': col('CHCSCNC1').isin([2, 7, 9]) | col('CHCOCNC1').isin([2, 7, 9]) | col('CHCSCNC1').isNull() | col('CHCOCNC1').isNull() | col('CSRVTRT3').isin([1, 3, 4, 5, 7, 9]) | col('CSRVTRT3').isNull(),
    'CNCRAGE': col('CHCSCNC1').isin([2, 7, 9]) | col('CHCOCNC1').isin([2, 7, 9]) | col('CHCSCNC1').isNull() | col('CHCOCNC1').isNull() | col('CNCRDIFF').isin([7, 9]) | col('CNCRDIFF').isNull(),
    'CNCRTYP2': col('CHCSCNC1').isin([2, 7, 9]) | col('CHCOCNC1').isin([2, 7, 9]) | col('CHCSCNC1').isNull() | col('CHCOCNC1').isNull() | col('CNCRDIFF').isin([7, 9]) | col('CNCRDIFF').isNull(),
    # Cáncer prostático
    'PSATEST1': (col('_SEX') == 1) | (col('_AGE80') < 40),
    'PSASUGST': (col('_SEX') == 1) | (col('_AGE80') < 40),
    'PCSTALK1': (col('_SEX') == 1) | (col('_AGE80') < 40),
    'PCPSARS2': (col('_SEX') == 1) | (col('_AGE80') < 40) | col('PSATEST1').isin([2, 7, 9]) | col('PSATEST1').isNull(),
    # Dependencia funcional
    'CIMEMLOS': col('_AGE80') < 45,
    'CDHOUSE': (col('_AGE80') < 45) | col('CIMEMLOS').isin([2, 9]),
    'CDASSIST': (col('_AGE80') < 45) | col('CIMEMLOS').isin([2, 9]),
    'CDSOCIAL': (col('_AGE80') < 45) | col('CIMEMLOS').isin([2, 9]),
    'CDDISCUS': (col('_AGE80') < 45) | col('CIMEMLOS').isin([2, 9]),
    'CDHELP': (col('_AGE80') < 45) | col('CIMEMLOS').isin([2, 9]) | col('CDASSIST').isin([4, 5, 7, 9]),
    # Red de apoyo y cuidados
    'CRGVREL4': col('CAREGIV1').isin([2, 8, 7, 9]),
    'CRGVLNG1': col('CAREGIV1').isin([2, 8, 7, 9]),
    'CRGVEXPT': col('CAREGIV1').isin([2, 8, 7, 9]),
    'CRGVPER1': col('CAREGIV1').isin([2, 8, 7, 9]),
    'CRGVHOU1': col('CAREGIV1').isin([2, 8, 7, 9]),
    'CRGVHRS1': col('CAREGIV1').isin([2, 8, 7, 9]),
    'CRGVPRB3': col('CAREGIV1').isin([2, 8, 7, 9]),
    'CRGVALZD': (col('CAREGIV1').isin([2, 8, 7, 9])) | (col('CRGVPRB3') == 5),
    # Salud respiratoria
    'ASTHNOW': col('ASTHMA3').isin([2, 7, 9]),
    # Consumo de alcohol
    'ASBIALCH': col('CHECKUP1').isin([3, 4, 7, 8, 9, None]),
    'ASBIDRNK': col('CHECKUP1').isin([3, 4, 7, 8, 9, None]),
    'ASBIBING': col('CHECKUP1').isin([3, 4, 7, 8, 9, None]),
    'ASBIADVC': col('CHECKUP1').isin([3, 4, 7, 8, 9, None]),
    'ASBIRDUC': (col('CHECKUP1').isin([3, 4, 7, 8, 9, None]) |  col('ASBIALCH').isin([2, 7, 9, None]) | col('ASBIDRNK').isin([2, 7, 9, None]) | col('ASBIBING').isin([2, 7, 9, None])),
    # Armas de fuego
    'GUNLOAD': col('FIREARM5').isin([2, 7, 9, None]),
    'LOADULK2': (col('FIREARM5').isin([2, 7, 9, None]) | col('GUNLOAD').isin([2, 7, 9, None])),
}

# Aplicar condiciones para rellenar con ceros donde se cumpla la condición
for col_name, condition in conditions.items():
    df = df.withColumn(col_name, F.when(condition & df[col_name].isNull(), 0).otherwise(df[col_name]))

In [ ]:
<div style="font-family: 'Garamond'; font-size: 14px;">
    <normal>
Tratamiento específico para columnas que explícitamente necesitaremos la media y no la mediana

In [7]:
mean_value_children = df.groupBy().agg(F.mean('CHILDREN').alias('mean_CHILDREN')).collect()[0]['mean_CHILDREN']
df = df.fillna(mean_value_children, subset=['CHILDREN'])

In [8]:
mean_value_bmi5 = df.groupBy().agg(F.mean('_BMI5').alias('mean_bmi5')).collect()[0]['mean_bmi5']
df = df.fillna(mean_value_bmi5, subset=['_BMI5'])

<div style="font-family: 'Garamond'; font-size: 14px;">
    <normal> Todas las columnas restantes las completaremos con la mediana de la misma, la idea inicial era usar la moda pero no era viable con el método Imputer

In [9]:
# Ahora, se especifica las columnas a las que se desea aplicar el Imputer
input_columns = [col for col in df.columns if df.filter(df[col].isNull()).count() > 0]
output_columns = input_columns  # En este caso, sobrescribiremos las columnas originales

In [10]:
# Configurar el Imputer para imputar los valores nulos con la mediana
imputer = Imputer(
    strategy='median',
    inputCols=input_columns,
    outputCols=output_columns
)

In [11]:
# Aplicar el imputer y transformar el DataFrame
imputer_model = imputer.fit(df)
df = imputer_model.transform(df)

<div style="font-family: 'Garamond'; font-size: 14px;">
    <normal>
Revisamos si quedan valores nulos

In [13]:
# Imprime las columnas con valores nulos
for col in df.columns:
    null_count = df.where(df[col].isNull()).count()
    if null_count > 0:
        print(f"La columna '{col}' tiene {null_count} valores nulos.")

<div style="font-family: 'Garamond'; font-size: 14px;">
    <normal>
Y por último exportamos el csv

In [14]:
# Guardar el DataFrame como CSV
df.write.csv("C:\\Users\\patri\\OneDrive - UAB\\Documentos\\GitHub\\BRFSS-clustering\\datos\\BRFSS_Imputated_2022.csv", header=True, mode="overwrite")

<div style="font-family: 'Garamond'; font-size: 14px;">
    <normal>
        
Este notebook es parte de un proyecto enfocado en la aplicación de algoritmos de clustering a las encuestas BRFSS (Behavioral Risk Factor Surveillance System). El BRFSS es una encuesta telefónica realizada por los CDC (Centros para el Control y la Prevención de Enfermedades) que recopila datos sobre factores de riesgo de comportamiento entre adultos en los Estados Unidos. Este proyecto busca analizar y agrupar estos datos para identificar patrones y tendencias que puedan informar decisiones de políticas de salud pública.

El código comienza con la configuración del entorno de Spark, utilizando `SparkConf` y `SparkContext` para establecer parámetros como el nombre de la aplicación, configuraciones de memoria, y otros aspectos relevantes para la ejecución eficiente de operaciones de Spark. Se hace uso de `SparkSession`, una interfaz de alto nivel para Spark SQL y DataFrame API, facilitando la manipulación de datos estructurados.

La lectura de datos se realiza desde un archivo CSV que contiene datos limpios de la encuesta BRFSS. Este archivo es cargado en un DataFrame de Spark, donde cada columna es convertida al tipo de dato `double` para facilitar su análisis numérico. 

Una parte crucial del preprocesamiento es la gestión de valores nulos y atípicos, marcando específicamente valores como 9 o 99 (que en este contexto se interpretan como respuestas tipo "Refused") como nulos, excepto en columnas específicas que se excluyen de este tratamiento.

El notebook también detalla la aplicación de condiciones específicas a ciertas columnas para ajustar los datos antes de la imputación. Estas condiciones están relacionadas con aspectos de salud ginecológica, cáncer colorectal, hábitos de fumar, cáncer y manejo del dolor, entre otros. Esto permite una manipulación detallada y contextual de los datos, preparándolos para una imputación más precisa.

Para la imputación de valores nulos, se utiliza el `Imputer` de Spark ML, configurado para usar la mediana como estrategia de imputación. Esta elección ayuda a mantener la robustez del análisis frente a datos atípicos. Antes de aplicar el `Imputer`, se calculan y aplican valores medios para columnas específicas como `CHILDREN` y `_BMI5`, demostrando un enfoque de imputación personalizado antes de la aplicación del imputer generalizado.

Finalmente, el notebook concluye con la imputación de valores nulos utilizando la mediana para el resto de las columnas que aún contienen valores nulos, seguido de una revisión final para identificar columnas que aún puedan tener valores nulos. Los datos imputados se guardan en un archivo CSV, preparando el conjunto de datos para el análisis de clustering.

Este enfoque meticuloso para preparar los datos de BRFSS para el análisis de clustering no solo mejora la calidad de los datos sino que también asegura que las inferencias y patrones descubiertos sean representativos y útiles para informar decisiones relacionadas con la salud pública.